<a href="https://colab.research.google.com/github/SubhrajeetGhose/DeepLearning-CampusX/blob/main/CNN_Functional_API_Transfer_Learning_Age_Gender_Revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 330M/331M [00:15<00:00, 24.4MB/s]
100% 331M/331M [00:15<00:00, 22.1MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [6]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
len(age)

23708

In [8]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [9]:
df.shape

(23708, 3)

In [10]:
df.head()

,age,gender,img
0,43,0,43_0_0_20170120222409079.jpg.chip.jpg
1,26,1,26_1_2_20170116184248014.jpg.chip.jpg
2,25,0,25_0_0_20170119171142840.jpg.chip.jpg
3,1,0,1_0_4_20161221195419136.jpg.chip.jpg
4,24,1,24_1_0_20170117150515251.jpg.chip.jpg


In [11]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [12]:
train_df.shape

(20000, 3)

In [13]:
test_df.shape

(3708, 3)

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [23]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='raw' )
                                                    #class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='raw' )
                                                  #class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [27]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# ... (your existing code) ...

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=folder_path,
    x_col='img',
    y_col=['age','gender'],
    target_size=(200, 200),
    class_mode='raw',  # Use 'raw' to handle custom labels
    batch_size=32
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=folder_path,
    x_col='img',
    y_col=['age','gender'],
    target_size=(200, 200),
    class_mode='raw',
    batch_size=32
)


from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

model = Model(inputs=resnet.input,outputs=[output1,output2])

model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Found 20000 validated image filenames.
Found 3708 validated image filenames.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:678: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 248s 371ms/step - age_mae: 17.4267 - loss: 17.4267 - val_age_mae: 16.4097 - val_loss: 16.4097
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 379ms/step - age_mae: 16.4929 - loss: 16.4929 - val_age_mae: 16.4082 - val_loss: 16.4082
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 234s 369ms/step - age_mae: 16.4331 - loss: 16.4331 - val_age_mae: 16.4084 - val_loss: 16.4084
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 257s 364ms/step - age_mae: 16.3839 - loss: 16.3839 - val_age_mae: 16.4074 - val_loss: 16.4074
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 228s 362ms/step - age_mae: 16.3608 - loss: 16.3608 - val_age_mae: 16.4205 - val_loss: 16.4205
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 363ms/step - age_mae: 16.4658 - loss: 16.4658 - val_age_mae: 16.4069 - val_loss: 16.4069
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 224s 355ms/step - age_mae: 16.3634 - loss: 16.3634 - val_age_mae: 16.4122 - val_loss: 16.4122
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 220s 348ms/step - age_mae: 16.520

In [24]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [25]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [26]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [27]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [28]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [29]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:678: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 241s 361ms/step - age_mae: 17.4314 - loss: 17.4314 - val_age_mae: 16.6064 - val_loss: 16.6064
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 247s 347ms/step - age_mae: 16.4233 - loss: 16.4233 - val_age_mae: 16.4242 - val_loss: 16.4242
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 222s 353ms/step - age_mae: 16.4323 - loss: 16.4323 - val_age_mae: 16.4084 - val_loss: 16.4084
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 217s 344ms/step - age_mae: 16.4071 - loss: 16.4071 - val_age_mae: 16.4070 - val_loss: 16.4070
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 217s 344ms/step - age_mae: 16.3634 - loss: 16.3634 - val_age_mae: 16.4073 - val_loss: 16.4073
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 342ms/step - age_mae: 16.3138 - loss: 16.3138 - val_age_mae: 16.4071 - val_loss: 16.4071
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 265s 348ms/step - age_mae: 16.3909 - loss: 16.3909 - val_age_mae: 16.4074 - val_loss: 16.4074
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 215s 341ms/step - age_mae: 16.469